# Аналитический рекомендатель по содержанию вакансий 

Разработаем механизм рекомендаций с использованием NLTK, который поможет соискателям выбирать предпочтительную работу на основе заявок.

В процессе узнаем, как лемматизация, стемминг и векторизация используются для обработки данных и получения лучшего результата. 

Поехали!

In [ ]:
#Импортируем библиотеки

import pandas as pd
import numpy as np

#Загрузим файл

final_jobs = pd.read_csv("../input/Combined_Jobs_Final.csv")

#Посмотрим первые 5 строк набора данных

final_jobs.head()

In [ ]:
# Список всех столбцов, которые присутствуют в наборе данных

list(final_jobs) 

In [ ]:
import seaborn as sns
sns.heatmap(final_jobs.isnull(), cbar=False); 

In [ ]:
print(final_jobs.shape)
final_jobs.isnull().sum()

Из приведенного выше списка мы видим, что существует много значений NaN. Выполним очистку данных для каждого столбца. 

## Объединим столбцы в рабочий корпус


In [ ]:
# Возьмём только необходимые для анализа столбцы
cols = list(['Job.ID']+['Slug']+['Title']+['Position']+ ['Company']+['City']+['Employment.Type']+['Education.Required']+['Job.Description'])
final_jobs =final_jobs[cols]
final_jobs.columns = ['Job.ID','Slug', 'Title', 'Position', 'Company','City', 'Empl_type','Edu_req','Job_Description']
final_jobs.head() 

In [ ]:
#Проверим нулевые значения
final_jobs.isnull().sum()

In [ ]:
#Посмотрим пропущенные значения в колонке Город
nan_city = final_jobs[pd.isnull(final_jobs['City'])]
print(nan_city.shape)
nan_city.head()

In [ ]:
nan_city.groupby(['Company'])['City'].count() 

Видно, что всего 9 городов компаний имеют значение NaN, поэтому вручную добавим их головные офисы, покопавшись в сети. 


In [ ]:
#Заменим пропущенные значения локацией штаб-квартир

final_jobs['Company'] = final_jobs['Company'].replace(['Genesis Health Systems'], 'Genesis Health System')

final_jobs.loc[final_jobs.Company == 'CHI Payment Systems', 'City'] = 'Illinois'
final_jobs.loc[final_jobs.Company == 'Academic Year In America', 'City'] = 'Stamford'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing ', 'City'] = 'Urbandale'
final_jobs.loc[final_jobs.Company == 'Driveline Retail', 'City'] = 'Coppell'
final_jobs.loc[final_jobs.Company == 'Educational Testing Services', 'City'] = 'New Jersey'
final_jobs.loc[final_jobs.Company == 'Genesis Health System', 'City'] = 'Davennport'
final_jobs.loc[final_jobs.Company == 'Home Instead Senior Care', 'City'] = 'Nebraska'
final_jobs.loc[final_jobs.Company == 'St. Francis Hospital', 'City'] = 'New York'
final_jobs.loc[final_jobs.Company == 'Volvo Group', 'City'] = 'Washington'
final_jobs.loc[final_jobs.Company == 'CBS Healthcare Services and Staffing', 'City'] = 'Urbandale'

In [ ]:
final_jobs.isnull().sum()

In [ ]:
#Тип занятости NA от Uber, поэтому я предполагаю, что это неполный или полный рабочий день. 

nan_emp_type = final_jobs[pd.isnull(final_jobs['Empl_type'])]
print(nan_emp_type)

In [ ]:
#Заменим NA значения на "Неполный / Полный рабочий день"
final_jobs['Empl_type'] = final_jobs['Empl_type'].fillna('Full-Time/Part-Time')
final_jobs.groupby(['Empl_type'])['Company'].count()
list(final_jobs)

#   Объединение 

#### Объединение столбцов position, company, city, empl_type и jobDesc

In [ ]:
final_jobs["pos_com_city_empType_jobDesc"] = final_jobs["Position"].map(str) + " " + final_jobs["Company"] +" "+ final_jobs["City"]+ " "+final_jobs['Empl_type']+" "+final_jobs['Job_Description']
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
#Удалим ненужные символы между словами, разделенными пробелами во всех столбцах, 
#чтобы сделать данные эффективными 

final_jobs['pos_com_city_empType_jobDesc'] = final_jobs['pos_com_city_empType_jobDesc'].str.replace('[^a-zA-Z \n\.]'," ")
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
#Преобразуем все символы в нижний регистр 

final_jobs['pos_com_city_empType_jobDesc'] = final_jobs['pos_com_city_empType_jobDesc'].str.lower() 
final_jobs.pos_com_city_empType_jobDesc.head()

In [ ]:
final_all = final_jobs[['Job.ID', 'pos_com_city_empType_jobDesc']]
final_all = final_all.fillna(" ")

final_all.head()

А вот и важная концепция **Стоп-слов**. 
Стоп-слова — это слова естественного языка, которые имеют очень мало значения, такие как «and», «the», «a», «an» и подобные.
Используем NLP, где NLTK (Natural Language Toolkit) используется для игнорирования слов.
Текст может содержать такие стоп-слова, как «the», «is», «are». Стоп-слова можно отфильтровать из обрабатываемого текста. Универсального списка стоп-слов в исследовании nlp не существует, однако модуль nltk содержит список стоп-слов. 

Следующий используемый здесь пакет — это стемминг. Идея стемминга — это своего рода метод нормализации. Многие варианты слов несут одно и то же значение, кроме случаев, когда используется время. Итак, чтобы очистить пространство, мы используем метод стемминга, и один из используемых здесь пакетов — PorterStemmer. 

In [ ]:
print(final_all.head(1))

 Импортируем стоп-слова из nltk.corpus.
  ##  NLTK - это аббревиатура от Natural Language Toolkit. 
Удаляет такие стоп-слова: the, is, and etc..

In [ ]:
pos_com_city_empType_jobDesc = final_all['pos_com_city_empType_jobDesc']

#Удаление стоп-слов и применение PorterStemmer 

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer =  PorterStemmer()
stop = stopwords.words('english')
only_text = pos_com_city_empType_jobDesc.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
only_text.head()


Разбиение каждого слова на строку через пробел. 

In [ ]:
only_text = only_text.apply(lambda x : filter(None,x.split(" ")))
print(only_text.head())

Здесь **стемминг** в основном используется для удаления суффиксов и общих слов, которые повторяются и разделяются запятыми. for y in x означает для каждого слова (y) в общем списке (x) 

In [ ]:
only_text = only_text.apply(lambda x : [stemmer.stem(y) for y in x])
print(only_text.head())

В приведенном выше коде мы разделили каждую букву в слове через запятую, теперь на этом шаге мы соединяем слова (x) 


In [ ]:
only_text = only_text.apply(lambda x : " ".join(x))
print(only_text.head())

In [ ]:
#Добавление избранного столбца обратно в pandas

final_all['text']= only_text

#Поскольку добавили новый столбец, выполнив все операции с использованием лямбда-функции, 
#удалим ненужный столбец 

final_all = final_all.drop("pos_com_city_empType_jobDesc", 1)

list(final_all)
final_all.head()

In [ ]:
#сохранить этот файл для резервного копирования 
#final_all.to_csv("job_data.csv", index=True)

# TF-IDF (Term Frequency - Inverse Document Frequency) 
Этот метод также называется нормализацией.
TF — сколько раз конкретное слово встречается в одном документе.
IDF — уменьшает масштаб слов, которые часто встречаются в документах. 

In [ ]:
#Инициализация tfidf векторизатора

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf_vectorizer = TfidfVectorizer()

tfidf_jobid = tfidf_vectorizer.fit_transform((final_all['text'])) #подгонка и преобразование вектора 
tfidf_jobid

# Корпус запросов пользователей
Возьмём другой набор данных, который называется Job_Views

In [ ]:
#Рассмотрим новый набор данных и примем во внимание наборов данных job_view 
#интересуют: должность, опыт кандидата для создания запроса, который подал заявку на работу

job_view = pd.read_csv("../input/Job_Views.csv")
job_view.head()


In [ ]:
sns.heatmap(job_view.isnull(), cbar=False);

In [ ]:
#Подмножество только необходимых столбцов, без учёта ненужных

job_view = job_view[['Applicant.ID', 'Job.ID', 'Position', 'Company','City']]

job_view["pos_com_city"] = job_view["Position"].map(str) + "  " + job_view["Company"] +"  "+ job_view["City"]

job_view['pos_com_city'] = job_view['pos_com_city'].str.replace('[^a-zA-Z \n\.]',"")

job_view['pos_com_city'] = job_view['pos_com_city'].str.lower()

job_view = job_view[['Applicant.ID','pos_com_city']]

job_view.head()


### Experience
Возьмём опыт всех соискателей, подавших заявки на вакансию, и сравниваем интересующие нас места с вакансиями, которые присутствовали в наших предыдущих данных. 

In [ ]:
#Experience
exper_applicant = pd.read_csv("../input/Experience.csv")
exper_applicant.head()

In [ ]:
sns.heatmap(exper_applicant.isnull(), cbar=False);

In [ ]:
#Возьмём только Position

exper_applicant = exper_applicant[['Applicant.ID','Position.Name']]

#Почистим текст

exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.replace('[^a-zA-Z \n\.]',"")

exper_applicant.head()
list(exper_applicant)

In [ ]:
exper_applicant['Position.Name'] = exper_applicant['Position.Name'].str.lower()
exper_applicant.head(10)

In [ ]:
exper_applicant =  exper_applicant.sort_values(by='Applicant.ID')
exper_applicant = exper_applicant.fillna(" ")
exper_applicant.head(20)


Для applicant_id 10001 описание должности отображается как Nan в первых трех строках, поэтому эти наблюдения будут удалены, и не будут учитываться в наборе данных.

In [ ]:
#Добавим одинаковые строки в одну

exper_applicant = exper_applicant.groupby('Applicant.ID', sort=False)['Position.Name'].apply(' '.join).reset_index()
exper_applicant.head(20)

### Position of Interest

In [ ]:
#Position of interest

poi =  pd.read_csv("../input/Positions_Of_Interest.csv", sep=',')
poi = poi.sort_values(by='Applicant.ID')
poi.head()

In [ ]:
sns.heatmap(poi.isnull(), cbar=False);

In [ ]:
# Нет необходимости в создании и обновлении, т.к. 
# у нас нет дедлайна, поэтому отбросим ненужное

poi = poi.drop('Updated.At', 1)
poi = poi.drop('Created.At', 1)

#Почистим текст

poi['Position.Of.Interest'] = poi['Position.Of.Interest'].str.replace('[^a-zA-z \n\.]',"")
poi['Position.Of.Interest'] = poi['Position.Of.Interest'].str.lower()
poi = poi.fillna(" ")
poi.head(20)

In [ ]:
poi = poi.groupby('Applicant.ID', sort=True)['Position.Of.Interest'].apply(' '.join).reset_index()
poi.head()

### Слияние

In [ ]:
#Сольём jobs и experience 

out_joint_jobs = job_view.merge(exper_applicant, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
print(out_joint_jobs.shape)
out_joint_jobs = out_joint_jobs.fillna(' ')
out_joint_jobs = out_joint_jobs.sort_values(by='Applicant.ID')
out_joint_jobs.head()

In [ ]:
#Сольём position of interest с существующим датафреймом

joint_poi_exper_view = out_joint_jobs.merge(poi, left_on='Applicant.ID', right_on='Applicant.ID', how='outer')
joint_poi_exper_view = joint_poi_exper_view.fillna(' ')
joint_poi_exper_view = joint_poi_exper_view.sort_values(by='Applicant.ID')
joint_poi_exper_view.head()

In [ ]:
#Объединим все столбцы

joint_poi_exper_view["pos_com_city1"] = joint_poi_exper_view["pos_com_city"].map(str) + joint_poi_exper_view["Position.Name"] +" "+ joint_poi_exper_view["Position.Of.Interest"]

joint_poi_exper_view.head()

In [ ]:
final_poi_exper_view = joint_poi_exper_view[['Applicant.ID','pos_com_city1']]
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view.columns = ['Applicant_id','pos_com_city1']
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view = final_poi_exper_view.sort_values(by='Applicant_id')
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.replace('[^a-zA-Z \n\.]',"")
final_poi_exper_view.head()


In [ ]:
final_poi_exper_view['pos_com_city1'] = final_poi_exper_view['pos_com_city1'].str.lower()
final_poi_exper_view.head()

In [ ]:
final_poi_exper_view = final_poi_exper_view.reset_index(drop=True)
final_poi_exper_view.head()

Возьмём рандомную строку

In [ ]:
#Берём пользователя
u = 6945
index = np.where(final_poi_exper_view['Applicant_id'] == u)[0][0]
user_q = final_poi_exper_view.iloc[[index]]
user_q

# Используем модель векторного пространства (косинусное подобие)
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

**Теория**

Нам нужно найти какое-то сходство между описанием должности и набором резюме. 
Косинусное сходство между выборками по X и Y: вычисляет подобие как нормализованное скалярное произведение X и Y:

     K (X, Y) = <X, Y> / (|| X || * || Y ||)
Результатом является косинусное подобие, которое инвариантно к масштабированию и ограничивает значение от -1 до 1. Значение косинуса 0 означает, что два вектора расположены под углом 90 градусов друг к другу (ортогональны) и не имеют совпадений. Чем ближе значение косинуса к 1, тем меньше угол и больше совпадение между векторами. 
В общем, cos θ указывает на подобие с точки зрения направления векторов. Это остается в силе по мере увеличения числа измерений, следовательно, cos θ является полезной мерой в многомерном пространстве. 

In [ ]:
#Создание tf-idf запроса соискателя и вычисление его косинусного сходства с работой 

from sklearn.metrics.pairwise import cosine_similarity
user_tfidf = tfidf_vectorizer.transform(user_q['pos_com_city1'])
output = map(lambda x: cosine_similarity(user_tfidf, x),tfidf_jobid)


In [ ]:
output2 = list(output)

In [ ]:
#Берём job id's из рекоммендаций

top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:50]
recommendation = pd.DataFrame(columns = ['ApplicantID', 'JobID'])

count = 0
for i in top:
    recommendation.at[count, 'ApplicantID'] = u
    recommendation.at[count,'JobID'] = final_all['Job.ID'][i]
    count += 1

****Оценка косинусного сходства****

In [ ]:
recommendation

In [ ]:
#Получение job id's и их данных

nearestjobs = recommendation['JobID']
job_description = pd.DataFrame(columns = ['JobID','text'])
for i in nearestjobs:
    index = np.where(final_all['Job.ID'] == i)[0][0]    
    job_description.at[count, 'JobID'] = i
    job_description.at[count, 'text'] = final_all['text'][index]
    count += 1

In [ ]:
#Выведем работу, соответствующую запросам

job_description

In [ ]:
job_description.to_csv("recommended_content.csv")

In [ ]:
final_all.to_csv("job_data.csv", index=False)